## Question 1.
Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

   Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

link for the data http://ec.europa.eu/eurostat/tgm/mapToolClosed.do?tab=map&init=1&plugin=1&language=en&pcode=tepsr_wc170&toolbox=types

In [7]:
import numpy as np

In [8]:
import pandas as pd
import json
import folium

raw_data = pd.DataFrame()
raw_data = raw_data.from_csv("lfsa_urgaed_1.tsv", sep="\t")
raw_data.isnull().values.any()

False

In [9]:
raw_data.head()

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
"GEO,SEX,AGE,ISCED11,UNIT\TIME",,,,,,,,,,
"European Union (28 countries),Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.1,7.0,8.9,9.5,9.6,10.4,10.8,10.2,9.4,8.6
"European Union (27 countries),Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.1,7.0,8.9,9.5,9.6,10.4,10.8,10.2,9.4,8.5
"European Union (15 countries),Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.0,7.1,9.0,9.5,9.6,10.6,11.0,10.5,9.8,9.1
"Euro area (19 countries),Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.4,7.5,9.5,10.1,10.1,11.3,12.0,11.6,10.9,10.0
"Euro area (18 countries),Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.4,7.5,9.5,10.0,10.1,11.3,12.0,11.6,10.9,10.1


In [10]:
# keep most recent year
cols = list(raw_data.columns)
last_date = cols[-1]
transitory_1 = pd.DataFrame(raw_data[last_date].loc[raw_data[last_date].index.str.contains('Euro area|European Union')==False])
transitory_1

,2016
"GEO,SEX,AGE,ISCED11,UNIT\TIME",
"Belgium,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.8
"Bulgaria,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.6
"Czech Republic,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",4.0
"Denmark,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",6.2 b
"Germany (until 1990 former territory of the FRG),Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",4.1
"Estonia,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",6.8
"Ireland,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.9
"Greece,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",23.6
"Spain,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",19.6


In [11]:
transitory_2 = transitory_1.loc[transitory_1.index.str.contains('Total,From 15 to 74 years,All ISCED 2011 levels')]
transitory_2.loc[transitory_2[last_date].str.contains(':')==False]

,2016
"GEO,SEX,AGE,ISCED11,UNIT\TIME",
"Belgium,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.8
"Bulgaria,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.6
"Czech Republic,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",4.0
"Denmark,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",6.2 b
"Germany (until 1990 former territory of the FRG),Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",4.1
"Estonia,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",6.8
"Ireland,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.9
"Greece,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",23.6
"Spain,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",19.6


In [12]:
transit_list = list(transitory_2[last_date])
for idx, val in enumerate(transit_list):
    new_val = str(val).replace('b','').replace(' ','').replace('u','')
    transit_list[idx] = new_val

In [13]:
transit_list

['7.8',
 '7.6',
 '4.0',
 '6.2',
 '4.1',
 '6.8',
 '7.9',
 '23.6',
 '19.6',
 '10.1',
 '13.1',
 '11.7',
 '13.0',
 '9.6',
 '7.9',
 '6.3',
 '5.1',
 '4.7',
 '6.0',
 '6.0',
 '6.2',
 '11.2',
 '5.9',
 '8.0',
 '9.7',
 '8.8',
 '7.0',
 '4.8',
 '3.0',
 '4.7',
 '5.0',
 '23.7',
 '10.9']

In [14]:
transitory3 = transitory_2.drop(last_date,axis=1)
transitory3[last_date]= transit_list
transitory3=transitory3.astype(float)
transitory3.head(100)

,2016
"GEO,SEX,AGE,ISCED11,UNIT\TIME",
"Belgium,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.8
"Bulgaria,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.6
"Czech Republic,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",4.0
"Denmark,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",6.2
"Germany (until 1990 former territory of the FRG),Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",4.1
"Estonia,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",6.8
"Ireland,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",7.9
"Greece,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",23.6
"Spain,Total,From 15 to 74 years,All ISCED 2011 levels ,Percentage",19.6


In [15]:
clean_df = transitory3.copy()

def keep_country_name(idx):
    str_vec = idx.split(',')
    str_vec2= str_vec[0].split('(')
    return str_vec2[0]

# keep country name only
new_indexs = [keep_country_name(idx) for idx in clean_df.index]
clean_df = clean_df.set_index(np.arange(len(clean_df.index)))
clean_df['country']= new_indexs
#clean_df['country'] = clean_df['country'].str.replace('Former Yugoslav Republic of Macedonia','Macedonia')
clean_df

,2016,country
0,7.8,Belgium
1,7.6,Bulgaria
2,4.0,Czech Republic
3,6.2,Denmark
4,4.1,Germany
5,6.8,Estonia
6,7.9,Ireland
7,23.6,Greece
8,19.6,Spain
9,10.1,France


In [16]:
clean_df['country']= clean_df['country'].str.replace(' ','')
#clean_df['country']= clean_df['country'].str.replace('CzechRepublic','Czech Republic').replace('UnitedKingdom','United Kingdom').clean_df['country']= clean_df['country'].str.replace('CzechRepublic','Czech Republic').replace('UnitedKingdom','United Kingdom').replace('FormerYugoslavRepublicofMacedonia','The former Yugoslav Republic of Macedonia')


In [17]:

clean_df['country']= clean_df['country'].str.replace('CzechRepublic','Czech Republic').replace('UnitedKingdom','United Kingdom').replace('FormerYugoslavRepublicofMacedonia','The former Yugoslav Republic of Macedonia')

In [18]:
most_recent_euUMR = clean_df.copy()
most_recent_euUMR.index.is_unique

True

to get the json https://geojson-maps.ash.ms/
or 
Save the file as geojson: jsontxt = data.to_json()

In [26]:
#EU_country_geo_path = r'custom.geo.json'
#geo_json_data = json.load(open(EU_country_geo_path))
import topojson_lst

EU_country_topo_path = r'topojson/europe.topojson.json'
topo_json_data = json.load(open(EU_country_topo_path))
objects = topo_json_data["objects"]["arcs"]
objects

KeyError: 'arcs'

In [24]:
m_eu = folium.Map([+53,+10], tiles='cartodbpositron', zoom_start=4)
folium.TopoJson(objects).add_to(m_eu)

TypeError: __init__() missing 1 required positional argument: 'object_path'

https://github.com/python-visualization/folium/blob/v0.2.0/folium/utilities.py#L104

In [ ]:
thrs = pd.qcut(most_recent_euUMR[last_date],5)

In [ ]:
thrs

In [25]:
m_eu = folium.Map([+53,+10], tiles='cartodbpositron', zoom_start=4)
m_eu.choropleth(topo_data=topo_json_data, data=most_recent_euUMR.reset_index(),
             columns=['country',last_date],
             key_on='feature.properties.name_long',
             threshold_scale=[2.9,5.0,6.2,7.9,11.0,23.7],
             fill_color='YlGnBu', fill_opacity=0.7, line_opacity=0.3,
             legend_name='European unemployment rate in 2016')

TypeError: choropleth() got an unexpected keyword argument 'topo_data'

In [ ]:
m_eu

## Question 2.
Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

   Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.


## Question 3.
Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

## Question 4.
*BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [Röstigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?